# Google Gemini (and Gemma)

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/gemini.ipynb)

Ce bloc-note Jupyter explore brièvement les capacités de [Gemini](https://aistudio.google.com/) à travers son API en langage Python. D'une manière générale, on préférera utiliser les modèles de Google à travers les API de [LangChain](./langchain.ipynb) ou [LlamaIndex](./llamaindex.ipynb) pour les applications de génération augmentée de récupération (RAG).

Docs:
- https://ai.google.dev/
- https://github.com/google/generative-ai-python
- https://github.com/google/generative-ai-docs/tree/main/site/en/gemini-api/tutorials

In [73]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -qU python-dotenv
# !pip3 install -qU jax

# À installer dans tous les cas pour Google Colab et Github
if ('google.colab' in sys.modules) or ('CI' in os.environ):
    !pip3 install -qU google-generativeai
    !pip3 install -qU huggingface_hub
    !pip3 install -qU git+https://github.com/google-deepmind/gemma.git

## Chargement de la Clé pour Gemini

Il vous faut [obtenir de Google une clé](https://aistudio.google.com/app/prompts/new_chat/?hl=fr) pour exécuter ce bloc-note Jupyter. Ensuite, le chargement se fait soit à partir de l'environnement (fichier `.env`), soit à partir des secrets de Google Colab.

In [74]:
google_api_key = None
if 'google.colab' in sys.modules:
  from google.colab import userdata
  google_api_key = userdata.get('GOOGLE_API_KEY')
  huggingface_api_key = userdata.get('HUGGINGFACE_API_KEY')
else:
  from dotenv import load_dotenv, find_dotenv
  _ = load_dotenv(find_dotenv()) # lire le fichier .env local
  google_api_key  = os.environ['GOOGLE_API_KEY']
  huggingface_api_key = os.environ['HUGGINGFACE_API_KEY']

In [75]:
import google.generativeai as genai
genai.configure(api_key=google_api_key)

## Choix de modèle

In [76]:
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/learnlm-1.5-pro-experimental
models/embedding-001
models/text-embedding-004
models/aqa


In [77]:
model = genai.GenerativeModel(
  #model_name='gemini-1.5-flash',
  #model_name='gemini-1.5-flash-8b',
  #model_name='gemini-1.5-pro',
  model_name='gemini-2.0-flash-exp',
  generation_config={
    'temperature': 1,
    'top_p': 0.95,
    'top_k': 40,
    'max_output_tokens': 8192,
    'response_mime_type': 'text/plain',
  },
)

## Generation de texte

In [78]:
from IPython.display import display
from IPython.display import Markdown

response = model.generate_content("Quel age avait Napoléon quand il a été couronné empereur?")
display(Markdown(response.text))

Napoléon avait **35 ans** lorsqu'il a été couronné empereur des Français le 2 décembre 1804. Il était né le 15 août 1769.


## Conversation Chat

In [79]:
from IPython.display import display
from IPython.display import Markdown

chat_session = model.start_chat(history=[])
response = chat_session.send_message('Quand la guerre de 14-18 s\'est-elle terminée?')
display(Markdown(response.text))

La Première Guerre mondiale, aussi appelée la Guerre de 14-18, s'est terminée le **11 novembre 1918** avec la signature de l'armistice.


In [80]:
response = chat_session.send_message('Où s\'est-elle terminée?')
display(Markdown(response.text))

La Première Guerre mondiale s'est terminée par la signature de l'armistice dans un wagon de train, dans la **clairière de Rethondes**, en forêt de Compiègne, dans le nord de la France.

Il est important de noter que l'armistice n'est pas la fin officielle de la guerre. Les traités de paix, notamment le Traité de Versailles, ont été signés plus tard en 1919 pour officialiser la fin du conflit. Cependant, l'armistice du 11 novembre 1918 marque la fin des combats.


In [81]:
chat_session.history

[parts {
   text: "Quand la guerre de 14-18 s\'est-elle terminée?"
 }
 role: "user",
 parts {
   text: "La Première Guerre mondiale, aussi appelée la Guerre de 14-18, s\'est terminée le **11 novembre 1918** avec la signature de l\'armistice.\n"
 }
 role: "model",
 parts {
   text: "Où s\'est-elle terminée?"
 }
 role: "user",
 parts {
   text: "La Première Guerre mondiale s\'est terminée par la signature de l\'armistice dans un wagon de train, dans la **clairière de Rethondes**, en forêt de Compiègne, dans le nord de la France.\n\nIl est important de noter que l\'armistice n\'est pas la fin officielle de la guerre. Les traités de paix, notamment le Traité de Versailles, ont été signés plus tard en 1919 pour officialiser la fin du conflit. Cependant, l\'armistice du 11 novembre 1918 marque la fin des combats.\n"
 }
 role: "model"]

## Gemma

Pour accéder au modèle Gemma sur Hugging Face, il faut:
- Une clé Hugging Face obtenue sur https://huggingface.co/settings/tokens
- Accepter l'accord de licence de Gemma sur https://huggingface.co/google/gemma-2-2b-it

In [82]:
from huggingface_hub import snapshot_download
local_dir = snapshot_download(repo_id="google/gemma-2b-flax", token=huggingface_api_key)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [83]:
from gemma import params as params_lib
from gemma import transformer as transformer_lib
import sentencepiece as spm

vocab = spm.SentencePieceProcessor()
vocab.Load(f'{local_dir}/tokenizer.model')
params = params_lib.load_and_format_params(f'{local_dir}/2b')
transformer_config = transformer_lib.TransformerConfig.from_params(
    params=params,
    cache_size=1024
)
transformer = transformer_lib.Transformer(transformer_config)

In [84]:
from gemma import sampler as sampler_lib

sampler = sampler_lib.Sampler(
    transformer=transformer,
    vocab=vocab,
    params=params['transformer'],
)
# Gemma 2b comprend mal le français
prompt = ['How long lasted the first world war?']
response = sampler(input_strings=prompt, total_generation_steps=128)
print(response.text)
# Supprimer le sampler pour un nouveau prompt
del sampler

['\n\nAnswer:\n\nThe first world war lasted from\n1914 to 1918.']


In [ ]:
# Pour supprimer les modèles du cache
# !pip3 install -q -U "huggingface_hub[cli]"
# !huggingface-cli scan-cache
# !huggingface-cli delete-cache